# 🌱 MealMind: Multi-Agent Meal Planning Demo

This notebook demonstrates the complete MealMind system with all 5 agents working together.

## System Overview

MealMind uses 5 specialized agents:
1. **Profile Manager** - Manages family profiles and constraints
2. **Recipe Generator** - Generates recipes using Gemini AI
3. **Nutrition Compliance** - Validates recipes against dietary needs
4. **Schedule Optimizer** - Optimizes cooking time and ingredient reuse
5. **Grocery List Agent** - Creates organized shopping lists

## Setup

First, let's import all necessary modules and set up logging.

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

# Import agents and orchestrator
from agents import profile_manager, recipe_generator, nutrition_compliance, schedule_optimizer, grocery_agent
from orchestrator import orchestrator
from tools import nutrition_tool, cost_estimator

import json
from IPython.display import display, HTML, JSON

print("✓ All modules imported successfully!")
print(f"✓ Using Gemini: {recipe_generator.use_gemini}")

## Step 1: Create Household Profile

Let's create a household with 4 family members, each with different dietary needs.

In [ ]:
# Create household
household = profile_manager.create_household_profile(
    household_id="demo_notebook",
    cooking_time_max=45,
    appliances=["oven", "stove", "microwave", "blender"],
    budget_weekly=150.0,
    cuisine_preferences=["Mediterranean", "Asian", "Italian"]
)

print("✓ Household created")
print(f"  Max cooking time: {household['cooking_time_max_minutes']} min/day")
print(f"  Budget: ${household['budget_weekly']}/week")

In [ ]:
# Add family members with diverse needs
members = [
    {
        "name": "Alice",
        "age": 35,
        "health_conditions": ["PCOS"],
        "dietary_restrictions": ["low-GI"],
        "preferences": ["spicy", "asian cuisine"],
        "calorie_target": 1800
    },
    {
        "name": "Bob",
        "age": 37,
        "allergies": ["peanuts"],
        "dietary_restrictions": ["gluten-free"],
        "dislikes": ["mushrooms"],
        "calorie_target": 2200
    },
    {
        "name": "Charlie",
        "age": 12,
        "dietary_restrictions": ["vegetarian"],
        "preferences": ["pasta", "pizza"],
        "calorie_target": 1600
    },
    {
        "name": "Diana",
        "age": 10,
        "preferences": ["chicken", "rice"],
        "calorie_target": 1400
    }
]

for member_data in members:
    profile_manager.add_family_member(
        household_id="demo_notebook",
        **member_data
    )
    print(f"✓ Added {member_data['name']}")

print(f"\n✓ Household setup complete with {len(members)} family members")

## Step 2: View Constraints

Let's see all the constraints our system needs to respect.

In [ ]:
constraints = profile_manager.get_all_constraints("demo_notebook")

print("📋 Household Constraints:")
print(f"  Allergies: {', '.join(constraints['allergies']) or 'None'}")
print(f"  Dietary Restrictions: {', '.join(constraints['dietary_restrictions']) or 'None'}")
print(f"  Health Conditions: {', '.join(constraints['health_conditions']) or 'None'}")
print(f"  Dislikes: {', '.join(constraints['dislikes']) or 'None'}")

print("\n💊 Health Condition Guidelines:")
for condition, guidelines in constraints.get('health_guidelines', {}).items():
    print(f"  {condition.upper()}:")
    print(f"    Avoid: {', '.join(guidelines['avoid'])}")
    print(f"    Prefer: {', '.join(guidelines['prefer'])}")

## Step 3: Generate Meal Plan

Now let's use the orchestrator to generate a complete 3-day meal plan.

In [ ]:
print("Generating 3-day meal plan...")
print("This will:")
print("  1. Load household context")
print("  2. Generate recipes for each meal")
print("  3. Validate nutrition compliance")
print("  4. Optimize the schedule")
print("  5. Create grocery list\n")

meal_plan = orchestrator.quick_plan(
    household_id="demo_notebook",
    days=3
)

print("\n✓ Meal plan generated successfully!")

## Step 4: View Meal Plan

Let's display the weekly meal schedule.

In [ ]:
for day in meal_plan["weekly_plan"]:
    print(f"\n📅 DAY {day['day']}")
    print("-" * 70)
    
    for meal in day["meals"]:
        print(f"\n  {meal['meal_type'].upper()}: {meal['name']}")
        print(f"  Cuisine: {meal['cuisine']} | Time: {meal['cooking_time_minutes']} min")
        
        if meal.get("nutritional_info"):
            nutrition = meal["nutritional_info"]
            print(f"  Nutrition: {nutrition['calories']:.0f} cal | "
                  f"P: {nutrition['protein_g']:.0f}g | "
                  f"C: {nutrition['carbs_g']:.0f}g | "
                  f"F: {nutrition['fat_g']:.0f}g")

## Step 5: View Optimization Results

In [ ]:
optimization = meal_plan["optimization"]
cooking_stats = optimization["cooking_time_stats"]

print("⏱️  Cooking Time Analysis:")
print(f"   Average per day: {cooking_stats['average_per_day']:.0f} minutes")
print(f"   Total for week: {cooking_stats['total_minutes']:.0f} minutes")
print(f"   Range: {cooking_stats['min_day']:.0f} - {cooking_stats['max_day']:.0f} minutes")

print(f"\n📊 Optimization Score: {optimization['optimization_score']:.1f}/100")

if optimization["recommendations"]:
    print(f"\n💡 Recommendations:")
    for rec in optimization["recommendations"][:5]:
        print(f"   • {rec}")

## Step 6: View Grocery List

In [ ]:
grocery = meal_plan["grocery_list"]

print(f"📦 Grocery List Summary")
print(f"   Total Items: {grocery['total_items']}")
print(f"   Estimated Cost: ${grocery['total_estimated_cost']:.2f}")

print(f"\n   Categories:")
for category, items in grocery["items_by_category"].items():
    print(f"     • {category}: {len(items)} items")

if grocery["shopping_tips"]:
    print(f"\n💡 Shopping Tips:")
    for tip in grocery["shopping_tips"][:3]:
        print(f"   • {tip}")

## Step 7: Detailed Grocery List

In [ ]:
# Display first few items from each category
for category, items in grocery["items_by_category"].items():
    print(f"\n📦 {category.upper()}")
    for item in items[:3]:  # Show first 3 items
        print(f"   ☐ {item['name']:<30} {item['total_amount']:.0f} {item['unit']:<8} ${item['estimated_cost']:>6.2f}")
    if len(items) > 3:
        print(f"   ... and {len(items) - 3} more items")

## Step 8: Statistics & Insights

In [ ]:
# Calculate statistics
total_meals = sum(len(day["meals"]) for day in meal_plan["weekly_plan"])
validated = sum(
    1 for day in meal_plan["weekly_plan"]
    for meal in day["meals"]
    if meal.get("nutrition_validated")
)

print("📊 Plan Statistics:")
print(f"   Total Meals: {total_meals}")
print(f"   Validated: {validated}/{total_meals} ({validated/total_meals*100:.0f}%)")
print(f"   Optimization Score: {optimization['optimization_score']:.1f}/100")
print(f"   Total Cost: ${grocery['total_estimated_cost']:.2f}")
print(f"   Budget: $150.00")
print(f"   {'✓ Within budget!' if grocery['total_estimated_cost'] <= 150 else '⚠️ Over budget'}")

# Ingredient reuse
ingredient_reuse = optimization["ingredient_reuse_stats"]
reused = sum(1 for count in ingredient_reuse.values() if count >= 2)
total_unique = len(ingredient_reuse)
reuse_pct = (reused / total_unique * 100) if total_unique > 0 else 0

print(f"\n♻️  Ingredient Reuse: {reuse_pct:.0f}% ({reused}/{total_unique} ingredients)")

# Cooking time
avg_time = cooking_stats["average_per_day"]
print(f"\n⏱️  Average Cooking Time: {avg_time:.0f} min/day")
print(f"   {'✓ Within limit!' if avg_time <= 45 else '⚠️ Exceeds 45 min limit'}")

## Step 9: Top Reused Ingredients

In [ ]:
top_ingredients = sorted(
    ingredient_reuse.items(),
    key=lambda x: x[1],
    reverse=True
)[:10]

print("🔝 Most Used Ingredients:")
for ing, count in top_ingredients:
    print(f"   • {ing.title()}: {count}x")

## Step 10: Export Results

In [ ]:
# Save to file
output_file = Path.cwd().parent / "notebook_meal_plan.json"
with open(output_file, 'w') as f:
    json.dump(meal_plan, f, indent=2)

print(f"💾 Meal plan saved to: {output_file}")
print("\n✓ Demo complete!")

## Conclusion

This demo showcased:
- ✅ Multi-agent orchestration (5 agents)
- ✅ Constraint management (allergies, dietary restrictions, health conditions)
- ✅ Recipe generation with validation
- ✅ Nutrition compliance checking
- ✅ Schedule optimization
- ✅ Grocery list generation
- ✅ Cost estimation

**Next Steps:**
- Add your Gemini API key for live AI recipe generation
- Run evaluation tests to validate system performance
- Generate 7-day plans for full weekly planning